# Data Exploration

During this stage, our objective is to gain a deep understanding of the dataset we are working with. We aim to prepare our dataset for subsequent analysis and the development of Machine Learning models.
       
**Excel File Utilized : Delhi-House-Prices.xlsx**

**Imports**

In [1]:
import numpy as np
import pandas as pd
import os

**Creating a DataFrame from the Imported Excel File**

In [3]:
cwd = os.getcwd()
df = pd.read_excel(cwd + "/Delhi-House-Prices.xlsx")
df

,Tagline,Construction Status,Price,Area,Bathrooms
0,3 BHK in Sector 3 Dwarka Delhi,Ready to move,1.8 Cr,1900,3 Bathrooms
1,3 BHK in Sector 3 Dwarka Delhi,Ready to move,1.8 Cr,1900,2 Bathrooms
2,3 BHK in Sector 3 Dwarka Delhi,Ready to move,1.8 Cr,1900,2 Bathrooms
3,3 BHK in Sector 3 Dwarka Delhi,Ready to move,1.81 Cr,1900,3 Bathrooms
4,3 BHK in Sector 3 Dwarka Delhi,Ready to move,1.8 Cr,1900,3 Bathrooms
...,...,...,...,...,...
51975,NaN,Ready to move,25 L,600,NaN
51976,NaN,Ready to move,4 Cr,1800,NaN
51977,NaN,Ready to move,36.32 L,789,NaN
51978,NaN,Ready to move,48 L,1000,NaN


<hr>

**Extracting `BHK` from `Tagline`**

In [4]:
df['BHK'] = df['Tagline'].str.split(' BHK').str[0]

In [5]:
df['BHK'].unique()[:15]

array(['3', '2', '4', '1 RK  in Kalkaji Delhi', '5', '1',
       '  in Gurgaon Delhi Expressway Delhi', '  in Sainik Farm Delhi',
       '  in Ghitorni Delhi', '  in Vasant Kunj Delhi',
       '  in Chattarpur Delhi', '  in Saket Delhi',
       '  in Jaitpur Extension Part II Khadda Colony Delhi',
       '  in Talimabad Delhi', '  in Kalindi Kunj Mithapur Road Delhi'],
      dtype=object)

In [6]:
# Create a mask for rows where 'BHK' contains '1 RK' and set them to 1
mask = df['BHK'].str.contains('1 RK', case=False, na=False)
df.loc[mask, 'BHK'] = '1'

In [7]:
df['BHK'].unique()[:15]

array(['3', '2', '4', '1', '5', '  in Gurgaon Delhi Expressway Delhi',
       '  in Sainik Farm Delhi', '  in Ghitorni Delhi',
       '  in Vasant Kunj Delhi', '  in Chattarpur Delhi',
       '  in Saket Delhi',
       '  in Jaitpur Extension Part II Khadda Colony Delhi',
       '  in Talimabad Delhi', '  in Kalindi Kunj Mithapur Road Delhi',
       '  in Madanpur Khadar Delhi'], dtype=object)

In [8]:
# Create a mask for rows where 'BHK' contains 'in' and set them to NULL
mask = df['BHK'].str.contains('in', case=False, na=False)
df.loc[mask, 'BHK'] = 'NULL'

In [9]:
df['BHK'].unique()

array(['3', '2', '4', '1', '5', 'NULL', '8', '6', '7', '10', '28', '15',
       '9', '12', nan], dtype=object)

<hr>

**Extracting `Locality` from `Tagline`**

In [10]:
df['Locality'] = df['Tagline'].str.split(' in ').str[1].str.split(' Delhi').str[0]

In [11]:
df['Locality'].unique()[:25]

array(['Sector 3 Dwarka', 'Sector 4 Dwarka', 'Sector 12 Dwarka',
       'Sector 6 Dwarka', 'Sector 5 Dwarka', 'Sector 22 Dwarka',
       'Sector 23 Dwarka', 'Sector 10 Dwarka', 'Patparganj',
       'Sector 7 Dwarka', 'Sector 9 Dwarka', 'Geetanjali Enclave',
       'Sector 2 Dwarka', 'Sector 13 Dwarka', 'Sector 18A Dwarka',
       'Kalkaji', 'Mayur Vihar II', 'Rani Bagh Road', 'Sector-18 Dwarka',
       'Sector 11 Dwarka', 'Panchsheel Enclave', 'Sector 25 Rohini',
       'Sector 21 Rohini', 'Mansa Ram Park', 'Dwarka Mor'], dtype=object)

In [12]:
# Convert 'Locality' to proper case for uniformity
df['Locality'] = df['Locality'].apply(lambda x: x.title() if isinstance(x, str) else x)

In [13]:
df['Locality'].unique()[:25]

array(['Sector 3 Dwarka', 'Sector 4 Dwarka', 'Sector 12 Dwarka',
       'Sector 6 Dwarka', 'Sector 5 Dwarka', 'Sector 22 Dwarka',
       'Sector 23 Dwarka', 'Sector 10 Dwarka', 'Patparganj',
       'Sector 7 Dwarka', 'Sector 9 Dwarka', 'Geetanjali Enclave',
       'Sector 2 Dwarka', 'Sector 13 Dwarka', 'Sector 18A Dwarka',
       'Kalkaji', 'Mayur Vihar Ii', 'Rani Bagh Road', 'Sector-18 Dwarka',
       'Sector 11 Dwarka', 'Panchsheel Enclave', 'Sector 25 Rohini',
       'Sector 21 Rohini', 'Mansa Ram Park', 'Dwarka Mor'], dtype=object)

<hr>

**Extracting numerical values from `Price`**

In [14]:
df['Price'].unique()

array(['1.8  Cr', '1.81  Cr', '1.79  Cr', ..., '19.49  L', '15.49  L',
       '36.32  L'], dtype=object)

In [15]:
def convert_price(price_str):
    price_str = price_str.replace(',', '').strip()
    
    parts = price_str.split()
    
    if len(parts) == 2:
        value, unit = (parts[0], parts[1])
    else:
        if 'L' in price_str or 'Cr' in price_str:
            value, unit = price_str.split('L') if 'L' in price_str else price_str.split('Cr')
        else:
            return price_str
    
    conversion_factors = {'L': 1, 'Cr': 100}
    return float(value) * conversion_factors.get(unit, 1)

df['Price (in Lakhs)'] = df['Price'].apply(convert_price)
df

,Tagline,Construction Status,Price,Area,Bathrooms,BHK,Locality,Price (in Lakhs)
0,3 BHK in Sector 3 Dwarka Delhi,Ready to move,1.8 Cr,1900,3 Bathrooms,3,Sector 3 Dwarka,180.0
1,3 BHK in Sector 3 Dwarka Delhi,Ready to move,1.8 Cr,1900,2 Bathrooms,3,Sector 3 Dwarka,180.0
2,3 BHK in Sector 3 Dwarka Delhi,Ready to move,1.8 Cr,1900,2 Bathrooms,3,Sector 3 Dwarka,180.0
3,3 BHK in Sector 3 Dwarka Delhi,Ready to move,1.81 Cr,1900,3 Bathrooms,3,Sector 3 Dwarka,181.0
4,3 BHK in Sector 3 Dwarka Delhi,Ready to move,1.8 Cr,1900,3 Bathrooms,3,Sector 3 Dwarka,180.0
...,...,...,...,...,...,...,...,...
51975,NaN,Ready to move,25 L,600,NaN,NaN,NaN,25.0
51976,NaN,Ready to move,4 Cr,1800,NaN,NaN,NaN,400.0
51977,NaN,Ready to move,36.32 L,789,NaN,NaN,NaN,36.32
51978,NaN,Ready to move,48 L,1000,NaN,NaN,NaN,48.0


<hr>

**Extracting numerical values from `Bathrooms`**

In [16]:
df['Bathroom'] = df['Bathrooms'].str.split(' Bathrooms').str[0]
df

,Tagline,Construction Status,Price,Area,Bathrooms,BHK,Locality,Price (in Lakhs),Bathroom
0,3 BHK in Sector 3 Dwarka Delhi,Ready to move,1.8 Cr,1900,3 Bathrooms,3,Sector 3 Dwarka,180.0,3
1,3 BHK in Sector 3 Dwarka Delhi,Ready to move,1.8 Cr,1900,2 Bathrooms,3,Sector 3 Dwarka,180.0,2
2,3 BHK in Sector 3 Dwarka Delhi,Ready to move,1.8 Cr,1900,2 Bathrooms,3,Sector 3 Dwarka,180.0,2
3,3 BHK in Sector 3 Dwarka Delhi,Ready to move,1.81 Cr,1900,3 Bathrooms,3,Sector 3 Dwarka,181.0,3
4,3 BHK in Sector 3 Dwarka Delhi,Ready to move,1.8 Cr,1900,3 Bathrooms,3,Sector 3 Dwarka,180.0,3
...,...,...,...,...,...,...,...,...,...
51975,NaN,Ready to move,25 L,600,NaN,NaN,NaN,25.0,NaN
51976,NaN,Ready to move,4 Cr,1800,NaN,NaN,NaN,400.0,NaN
51977,NaN,Ready to move,36.32 L,789,NaN,NaN,NaN,36.32,NaN
51978,NaN,Ready to move,48 L,1000,NaN,NaN,NaN,48.0,NaN


<hr>

**Creating new DF to be used for Data Cleaning**

In [17]:
df2 = df[['Locality', 'Area', 'BHK', 'Bathroom', 'Price (in Lakhs)']]
df2

,Locality,Area,BHK,Bathroom,Price (in Lakhs)
0,Sector 3 Dwarka,1900,3,3,180.0
1,Sector 3 Dwarka,1900,3,2,180.0
2,Sector 3 Dwarka,1900,3,2,180.0
3,Sector 3 Dwarka,1900,3,3,181.0
4,Sector 3 Dwarka,1900,3,3,180.0
...,...,...,...,...,...
51975,NaN,600,NaN,NaN,25.0
51976,NaN,1800,NaN,NaN,400.0
51977,NaN,789,NaN,NaN,36.32
51978,NaN,1000,NaN,NaN,48.0


<hr>

**Export the Explored `df2`**

In [18]:
cwd = os.getcwd()
df2.to_excel(cwd + "/Explored-Delhi-Prices.xlsx", index = False)